# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
import os
from dotenv import load_dotenv
from huggingface_hub import login

# Load environment variables from .env file
load_dotenv()

# Get HF_TOKEN from environment variables
hf_token = os.environ.get("HF_TOKEN")

# Login to Hugging Face with the token
if hf_token:
    login(token=hf_token)
else:
    print("HF_TOKEN not found in environment variables. Please login manually:")
    login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

train_dataset = ds["train"]
test_dataset = ds["test"]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
from datasets import load_dataset




def process_dataset(sample):
    # TODO: 🐢 Convert the sample into a chat format
    # use the tokenizer's method to apply the chat template


    # Extract the messages from the sample
    messages = sample['messages']
        
    # Apply the chat template to create a formatted conversation
    formatted_conversation = tokenizer.apply_chat_template(
        messages, 
        tokenize=False  # We want the text, not tokens
    )
        
    # Add the formatted conversation to the sample
    sample['formatted_conversation'] = formatted_conversation

    return sample


# ds = ds.map(process_dataset)

In [6]:
def formatting_func(example):
    messages = example['messages']
    # Apply the chat template to create a formatted conversation
    formatted_conversation = tokenizer.apply_chat_template(
        messages, 
        tokenize=False  # We want the text, not tokens
    )
    # return text
    return formatted_conversation

In [7]:
formatting_func(train_dataset[0])

"<|im_start|>user\nHi there<|im_end|>\n<|im_start|>assistant\nHello! How can I help you today?<|im_end|>\n<|im_start|>user\nI'm looking for a beach resort for my next vacation. Can you recommend some popular ones?<|im_end|>\n<|im_start|>assistant\nSome popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.<|im_end|>\n<|im_start|>user\nThat sounds great. Are there any resorts in the Caribbean that are good for families?<|im_end|>\n<|im_start|>assistant\nYes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.<|im_end|>\n<|im_start|>user\nOkay, I'll look into those. Thanks for the recommendations!<|im_end|>\n<|im_start|>assistant\nYou're welcome. I hope you find the perfect resort for your vacation.<|im_end|>\n"

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [9]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [10]:
SFTConfig??

Init signature:
SFTConfig(
    output_dir: Optional[str] = None,
    overwrite_output_dir: bool = False,
    do_train: bool = False,
    do_eval: bool = False,
    do_predict: bool = False,
    eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no',
    prediction_loss_only: bool = False,
    per_device_train_batch_size: int = 8,
    per_device_eval_batch_size: int = 8,
    per_gpu_train_batch_size: Optional[int] = None,
    per_gpu_eval_batch_size: Optional[int] = None,
    gradient_accumulation_steps: int = 1,
    eval_accumulation_steps: Optional[int] = None,
    eval_delay: Optional[float] = 0,
    torch_empty_cache_steps: Optional[int] = None,
    learning_rate: float = 2e-05,
    weight_decay: float = 0.0,
    adam_beta1: float = 0.9,
    adam_beta2: float = 0.999,
    adam_epsilon: float = 1e-08,
    max_grad_norm: float = 1.0,
    num_train_epochs: float = 3.0,
    max_steps: int = -1,
    lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType,

In [11]:
SFTTrainer??

Init signature:
SFTTrainer(
    model: Union[str, torch.nn.modules.module.Module, transformers.modeling_utils.PreTrainedModel],
    args: Union[trl.trainer.sft_config.SFTConfig, transformers.training_args.TrainingArguments, NoneType] = None,
    data_collator: Optional[transformers.data.data_collator.DataCollator] = None,
    train_dataset: Union[datasets.arrow_dataset.Dataset, datasets.iterable_dataset.IterableDataset, NoneType] = None,
    eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None,
    processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.image_processing_utils.BaseImageProcessor, transformers.feature_extraction_utils.FeatureExtractionMixin, transformers.processing_utils.ProcessorMixin, NoneType] = None,
    compute_loss_func: Optional[Callable] = None,
    compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], dict]] = None,
    callbacks: Optio

In [12]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [13]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=1,  # Set according to your GPU memory capacity
    learning_rate=1e-4,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=100,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    max_seq_length=128,  # Maximum sequence length for training
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    # tokenizer=tokenizer,
    processing_class=tokenizer,
    eval_dataset=ds["test"],
    formatting_func=formatting_func,
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:2243: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [14]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid de

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss
100,1.098100,1.149891
200,1.037700,1.101857
300,1.047100,1.100900
400,1.086200,1.091189
500,1.017700,1.051787
600,1.018400,1.031263
700,0.908400,1.017396
800,0.933500,1.005669
900,1.014600,0.996768
1000,0.945800,0.992318


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning:

In [15]:
trainer.push_to_hub(tags=finetune_tags)

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


training_args.bin:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

events.out.tfevents.1747210932.Admins-MacBook-Pro.local.40354.0:   0%|          | 0.00/41.7k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tuandunghcmut/SmolLM2-FT-MyDataset/commit/855a4eeaae60686b671dbd0773a0db684b527f2e', commit_message='End of training', commit_description='', oid='855a4eeaae60686b671dbd0773a0db684b527f2e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tuandunghcmut/SmolLM2-FT-MyDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='tuandunghcmut/SmolLM2-FT-MyDataset'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [16]:
test_dataset[-1]

{'full_topic': 'Music/Musical instruments/Unusual instruments',
 'messages': [{'content': 'Hi there', 'role': 'user'},
  {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
  {'content': "I'm interested in learning about unusual musical instruments. Can you tell me about one?",
   'role': 'user'},
  {'content': "One unusual instrument is the Theremin. It's an electronic instrument played without physical contact, using hand movements to control pitch and volume.",
   'role': 'assistant'},
  {'content': "That sounds interesting. What's another unusual instrument?",
   'role': 'user'},
  {'content': "Another unusual instrument is the Hang Drum. It's a modern, hand-played drum that produces a rich, resonant sound when played with the fingers.",
   'role': 'assistant'},
  {'content': "I'd love to hear what it sounds like.", 'role': 'user'}]}

In [17]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
# prompt = "Write a haiku about programming"
prompt = "What is the capital of France?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

outputs = model.generate(**inputs, max_new_tokens=256)

In [18]:
outputs[0]

tensor([    1,  4093,   198,  1780,   314,   260,  3575,   282,  4649,    47,
            2,   198, 33488,   314,   260,  3575,   282,  4649,    30,     2],
       device='mps:0')

In [19]:
formatted_prompt

'<|im_start|>user\nWhat is the capital of France?<|im_end|>\n'

In [20]:
# decoding the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("After training:")
print(decoded_output)

After training:
<|im_start|>user
What is the capital of France?<|im_end|>
France is the capital of France.<|im_end|>


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.